# A standalone example of ROOT geometry
We will see the 'simplest geometry' example, from ROOT Users Guide. This, altough very basilar, can tell us how the general structure works. 
## Creating the TGeoManager object
Every geometry must contain a TGeoManager instance: this will allow us to navigate in our geometry, and to access all the information we need. We can then access it directly, or by the `gGeoManager` global variable

In [1]:
TGeoManager *mygeometry = new TGeoManager("world", "the simplest geometry");

Info in <TGeoManager::TGeoManager>: Geometry world, the simplest geometry created


## Creating our first volume
The basic brick of the geometry is the `volume`.Each volume represents an un-positioned pieces of the geometry puzzle. In order to be defined, it needs a `medium`, describing the material of our volume.

In [2]:
TGeoMaterial *mat = new TGeoMaterial("Vacuum",0,0,0);
TGeoMedium   *med = new TGeoMedium("Vacuum",1,mat);

We can now create our volume. We will use the most basic shape: the `Box`, a paralleliped with requres as input the three half-dimensions of the sizes:

In [3]:
float cm = 1.;
TGeoBBox *mybox = new TGeoBBox("mybox",10.*cm,10.*cm,10.*cm);
TGeoVolume *top = new TGeoVolume("Top",mybox,med);
gGeoManager->SetTopVolume(top);

Info in <TGeoManager::SetTopVolume>: Top volume is Top. Master volume is Top
Info in <TGeoNavigator::BuildCache>: --- Maximum geometry depth set to 100


Note: default ROOT units are in centimeters (altough units are arbitrary for actual geometry construction). FairShip follows that convention.

## Creating a composite shape
As an example of composite shape, we can build a box with a spherical cavity. We need then only two primitive shapes: a TGeoBBox for the actual box and a TGeoSphere for the cavity.

In [4]:
TGeoBBox * externalbox = new TGeoBBox("EB",2*cm,2*cm,2*cm);
TGeoSphere *internalsphere = new TGeoSphere("IS",0,1,0,180,0,360);

Applying the relative transformation, we can put the cavity wherever we want in our box. 
Please notice that the transformation must be named and registered, otherwise the boolean operation cannot recognize it, when we build the composite shape

In [5]:
TGeoTranslation *mytrans1 = new TGeoTranslation("mytrans1",-2 *cm, 0, 1 *cm);
mytrans1->SetName("mytrans1");
mytrans1->RegisterYourself();

Having defined the primitives and their relative transformation, we are ready to build the composite shape. Then we associate the shape to a new volume.

In [6]:
TGeoCompositeShape * hollowbox = new TGeoCompositeShape("hollowbox","EB-(IS:mytrans1)");
TGeoVolume *volcomposite = new TGeoVolume("volcomposite",hollowbox,med);

## Positioning daughter volumes
With expection of the `top` volume, TGeoVolume instances are positioned inside another one, according to a hierarchy based on `containment`. The method to do so is called AddNode(), called from the mother volume and accepting the daughter volume as argument.

In [7]:
top->AddNode(volcomposite,1,new TGeoTranslation(0,0,-5*cm));
top->AddNode(volcomposite,2,new TGeoTranslation(0,0,5*cm));

We are ready to close the geometry and to draw it:

In [8]:
volcomposite->SetLineColor(kRed);
gGeoManager->CloseGeometry();

Info in <TGeoManager::CheckGeometry>: Fixing runtime shapes...
Info in <TGeoManager::CheckGeometry>: ...Nothing to fix
Info in <TGeoManager::CloseGeometry>: Counting nodes...
Info in <TGeoManager::Voxelize>: Voxelizing...
Info in <TGeoManager::CloseGeometry>: Building cache...
Info in <TGeoManager::CountLevels>: max level = 1, max placements = 2
Info in <TGeoManager::CloseGeometry>: 3 nodes/ 2 volume UID's in the simplest geometry
Info in <TGeoManager::CloseGeometry>: ----------------modeler ready----------------


Ready to have a look at our volume! Luckily, the notebook version of ROOT allows for interactive check of geometry displays, not only canvases! Let's just activate jsroot and draw our volume:

In [9]:
TCanvas *c = new TCanvas();
gGeoManager->GetTopVolume()->Draw();
c->Print("composite_example.png");

Info in <TCanvas::Print>: png file composite_example.png has been created
